# Neo4j GraphRAG Retriever Notebook

This notebook demonstrates how to use various retrievers and Cypher patterns with Neo4j GraphRAG for asset manager and cybersecurity risk retrieval.


In [ ]:
import os
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever, Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG
from dotenv import load_dotenv
import pandas as pd

from IPython.display import display, HTML

import textwrap

from IPython.core.display import HTML
display(HTML("<style>.output_area pre {white-space: pre-wrap; word-break: break-word;}</style>"))

# Load environment variables
load_dotenv()
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USER = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


In [ ]:
# --- Initialize LLM and Embedder ---
###model_params = {"response_format": {"type": "json_object"},  # use json_object formatting for best results
###                "temperature": 0  # turning temperature down for more deterministic results
###                }

llm = OpenAILLM(model_name='gpt-4o', api_key=openai_api_key)
embedder = OpenAIEmbeddings(api_key=openai_api_key)

In [5]:
# --- Initialize Vector Retriever ---

query = "What are the risks that Apple faces?"

vector_retriever = VectorRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    return_properties=['text'])

In [8]:
    # --- Simple Vector Search Diagnostic ---

result = vector_retriever.search(query_text=query, top_k=10)
result_table=pd.DataFrame([(item.metadata['score'], item.content[10:80], item.metadata['id']) for item in result.items], columns=['Score', 'Content', 'ID'])
result_table

,Score,Content,ID
0,0.916122,"the Company to potential liabilities, increase...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1492
1,0.912796,"relief against the Company, and has\nfrom time...",4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1986
2,0.910751,interruptions can harm or disrupt internationa...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1876
3,0.910599,in both\nfrequency and sophistication with in...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1946
4,0.909256,impact the Company\'s net sales to its indirec...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:2897
5,0.908752,intended to be inactive\ntextual references on...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1875
6,0.908371,conditions and adverse economic conditions can...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:2911
7,0.907104,interruption\naffecting such sources would exa...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1877
8,0.906647,perception and analysis of the relative benefi...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:1906
9,0.906570,sults.\nApple Inc. | 2023 Form 10-K | 25\nSect...,4:f86fa30c-6448-4779-bdfc-2f048de6bb86:3113


In [9]:
rag = GraphRAG(
    llm=llm,
    retriever=vector_retriever
)
#print(textwrap.fill(rag.search(query).answer, width=80))
print(rag.search(query).answer)

Apple faces several risks, including:

1. **Regulatory and Compliance Risks:** The company is subject to complex and changing laws and regulations worldwide, including those related to antitrust, privacy, data security, consumer protection, and environmental, social, and governance considerations. Failure to comply with these can lead to legal proceedings, increased costs, and adverse effects on the business.

2. **App Store Changes and Antitrust Investigations:** Apple has made changes to its App Store in response to legal and regulatory requirements and is subject to antitrust investigations around the world. Adverse findings could lead to significant fines and mandatory changes to its App Store business, impacting operations and financial performance.

3. **International Trade and Supply Chain Risks:** Restrictions on international trade, such as tariffs and import/export controls, can disrupt operations and supply chains. Tensions between countries, like the U.S. and China, may lea

In [11]:
# --- VectorCypherRetriever Example: Detailed Search with Context ---

query = "What are the top risk factors that Apple faces?"

detail_context_query = """
    with node
    MATCH (node)-[:FROM_DOCUMENT]-(doc:Document)-[:FILED]-(company:Company)-[:FACES_RISK]-(r:RiskFactor)
    RETURN node.text AS context,  collect(DISTINCT r) AS risks
    LIMIT 10
"""
vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    retrieval_query=detail_context_query
)

rag = GraphRAG(llm=llm, retriever=vector_cypher_retriever)
print(textwrap.fill(rag.search(query).answer, width=80))
# print(rag.search(query).answer)

Apple faces several top risk factors, including:  1. **Market Competition:**
Apple operates in highly competitive markets for its products, including
smartphones, tablets, and PCs, which puts pressure on pricing and innovation.
2. **Supply Chain Dependency:** Apple's reliance on a complex global supply
chain exposes it to risks such as manufacturing disruptions, geopolitical
tensions, and trade restrictions.  3. **Regulatory and Legal Challenges:** The
company faces ongoing scrutiny and potential legal challenges related to
antitrust issues, privacy laws, and various international regulations.  4.
**Technological Changes:** Rapid advancements in technology necessitate
continuous innovation, and failure to keep pace could affect Apple's market
position.  5. **Economic Conditions:** Global economic fluctuations can affect
consumer spending power, impacting sales and profitability for Apple’s premium
products.  6. **Cybersecurity Threats:** As a tech company, Apple is at risk
from cyberse

In [98]:
# --- Question Answering ---
print("\n--- Question Answering ---")
query = "Compare Apple's and Microsoft's market positions"
compare_query = """
    MATCH (c1:Company {name: 'Apple'})-[:HAS_METRIC]-(m1:FinancialMetric)
    MATCH (c2:Company {name: 'Microsoft'})-[:HAS_METRIC]-(m2:FinancialMetric)
    RETURN c1, c2, collect(m1) as apple_metrics, collect(m2) as microsoft_metrics
"""
compare_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name="chunkEmbeddings",
    embedder=embedder,
    retrieval_query=compare_query
)
compare_cypher_rag = GraphRAG(llm=llm, retriever=compare_cypher_retriever)
response = compare_cypher_rag.search(query, retriever_config={'top_k': 3})
print("\nComparison Response:")
print(response.answer)
response



--- Question Answering ---

Comparison Response:
As of the latest data available up to October 2023, Apple and Microsoft both hold strong positions in the technology sector, each excelling in different areas of the market.

Apple:
1. **Consumer Electronics Dominance**: Apple is a leader in consumer electronics, particularly with its iPhone, iPad, and Mac product lines. The company has a loyal customer base and is known for its premium pricing strategy, which contributes to high profit margins.

2. **Ecosystem and Services**: Apple's ecosystem, which includes the App Store, iCloud, Apple Music, and other services, creates a seamless user experience and generates a substantial portion of its revenue. The services segment has been growing steadily, providing recurring revenue streams.

3. **Innovation**: Apple is renowned for its innovation in hardware and design, consistently setting trends within the market. This is evidenced by its development of new product lines such as Apple Watch 

RagResultModel(answer="As of the latest data available up to October 2023, Apple and Microsoft both hold strong positions in the technology sector, each excelling in different areas of the market.\n\nApple:\n1. **Consumer Electronics Dominance**: Apple is a leader in consumer electronics, particularly with its iPhone, iPad, and Mac product lines. The company has a loyal customer base and is known for its premium pricing strategy, which contributes to high profit margins.\n\n2. **Ecosystem and Services**: Apple's ecosystem, which includes the App Store, iCloud, Apple Music, and other services, creates a seamless user experience and generates a substantial portion of its revenue. The services segment has been growing steadily, providing recurring revenue streams.\n\n3. **Innovation**: Apple is renowned for its innovation in hardware and design, consistently setting trends within the market. This is evidenced by its development of new product lines such as Apple Watch and AirPods, which h

In [19]:
# --- Text2CypherRetriever Example ---
from requests import session


print("\n--- Text2CypherRetriever Example ---")
text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema="(:Chunk)-[]-(:Document)-[]-(:Company),(:Company)-[]-(:RiskFactor)-[]-(:Company)-[]-(:AssetManager)")

query = "Show me all companies managed by BlackRock and the risk factors they face."
cypher_query = text2cypher_retriever.get_search_results(query)
print("\n--- Text2Cypher Output ---")
print("Original Query:", query)
print("Generated Cypher:\n", cypher_query.metadata["cypher"])



--- Text2CypherRetriever Example ---

--- Text2Cypher Output ---
Original Query: Show me all companies managed by BlackRock and the risk factors they face.
Generated Cypher:
 cypher
MATCH (am:AssetManager)-[]-(c:Company)-[]-(rf:RiskFactor)
WHERE am.name = 'BlackRock'
RETURN c.name AS Company, rf.name AS RiskFactor

